<a href="https://colab.research.google.com/github/praffuln/langchain/blob/master/langchain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openAI
!pip install wikipedia
!pip install huggingface_hub
!pip install google-search-results


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=a22230db64b5c101648926cc11f058466831b20349317f6e328eb865a34d39a6
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


# ***api keys***

follow credentials.txt !! not commited in git branch.

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'follow credentials.txt'
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "follow credentials.txt"
os.environ['SERPAPI_API_KEY'] = 'follow credentials.txt'

In [ ]:
import os

from langchain import PromptTemplate, HuggingFaceHub, LLMChain
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
#llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0})
llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature": 0.5}
)

question = "When was apple founded?"

print(llm(question))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


January 1976


PROMPTTemplate

In [ ]:
template = "When was {company} founded? "
prompt_template_company = PromptTemplate(input_variables =["company"], template = template)
prompt_template_company.format(company="apple")

'When was apple founded? '

**# Chain**

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template_company)
chain.run("ibm")

'1890'

Sequencial *chain*  

SimpleSequentialChain =======

 Input(India restaurent) -->(restaurent name) --> Output(Maharaja) ---> (GIVE ME food menu items) ---> lassi, tandoori, tea, samosa

 It just give only one output.

In [ ]:
cuisine_name_template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
cuisine_name_prompt = PromptTemplate(template=cuisine_name_template, input_variables= ["cuisine"])
cuisine_name_chain = LLMChain(llm=llm, prompt=cuisine_name_prompt)


items_template = "Suggest some menu item for {restaurant_name}. Return it as comma seprate list."
item_prompt = PromptTemplate(template=items_template, input_variables= ["restaurant_name"])
item_chain = LLMChain(llm=llm, prompt=item_prompt)


In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains= [cuisine_name_chain, item_chain])
response = chain.run(["chinese"])
response

"['shanghai','shanghai','shang"

SequentialChain *chain*  

SequentialChain =======

 Input(India restaurent) -->(restaurent name) --> Output(Maharaja) ---> (GIVE ME food menu items) ---> lassi, tandoori, tea, samosa

 It can give more than one output.

In [ ]:


cuisine_name_template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
cuisine_name_prompt = PromptTemplate(template=cuisine_name_template, input_variables= ["cuisine"])
cuisine_name_chain = LLMChain(llm=llm, prompt=cuisine_name_prompt, output_key="restaurent_name")

items_template = "Suggest some menu item for {restaurent_name}. Return it as comma seprate list."
item_prompt = PromptTemplate(template=items_template, input_variables= ["restaurant_name"])
item_chain = LLMChain(llm=llm, prompt=item_prompt, output_key="menu_items")

from langchain.chains import SequentialChain

chain = SequentialChain(chains= [cuisine_name_chain, item_chain], input_variables=['cuisine'],
                        output_variables=['menu_items'])
response = chain({'cuisine':'japani'})
response


{'cuisine': 'japani', 'menu_items': ',,,, '}

In [ ]:
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)
synopsis_chain = LLMChain(
    llm=llm, prompt=synopsis_prompt_template, output_key="synopsis"
)


template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)


overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})






> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': 'A young woman, suffering from tuberculosis, is raped by her',
 'review': 'a play that is too preoccupied with its own enigmatic nature to be fully appreciated'}

# *Agents*

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

tools = load_tools(["wikipedia", "llm-math"], llm = llm)
agent = initialize_agent(tools = tools, llm = llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)
test = agent.run('when was elon musk born ? what is his age in 2023 ?' )




> Entering new AgentExecutor chain...
wikipaedia elon musk age in 2023 is 66 years
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:wikipaedia elon musk age in 2023 is 66 years
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Thought:Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Thought:Thought:Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Thought:Thought:Thought:Thought:I now know the final
Observation: Invalid Format: Miss

**# SERPapi**

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

tools = load_tools(["serpapi"], llm = llm)
agent = initialize_agent(tools = tools, llm = llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)
test = agent.run('What was the GDP of US in 2022 ?' )




> Entering new AgentExecutor chain...
Search for "US GDP 2022" Action: Input "US GDP 2022" Observ
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer'
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer'
O

# ***MEMORY***

***ConversationBufferMemory***

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

template = "When was {company} founded? "
prompt_template_company = PromptTemplate(input_variables =["company"], template = template)
prompt_template_company.format(company="apple")

from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template_company, memory=memory) #here is magic
chain.run("ibm")

chain.memory
chain.run("apple")
chain.memory
print(chain.memory.buffer)

Human: ibm
AI: 1890
Human: apple
AI: September 1976


***ConversationChain***

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm =llm)
print(convo.prompt.template)


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
convo.run('who won first football world cup')

'Brazil'

In [ ]:
convo.run('what is 5 plus 5')

'5+5 is a family of quilt patterns with the same back and front, but different quilting designs.'

In [ ]:
print(convo.run('who was the captian of winning team'))

print(convo.memory.buffer)

Luiz Felipe Scolari
Human: who won first football world cup
AI: Brazil
Human: who was the captian of winning team
AI: Luiz Felipe Scolari
Human: who was the captian of winning team
AI: Luiz Felipe Scolari
Human: what is 5+5
AI: 5+5 is a family of quilt patterns with the same back and front, but different quilting designs.
Human: what is 5 plus 5
AI: 5+5 is a family of quilt patterns with the same back and front, but different quilting designs.
Human: who won first football world cup
AI: Brazil
Human: what is 5 plus 5
AI: 5+5 is a family of quilt patterns with the same back and front, but different quilting designs.
Human: who was the captian of winning team
AI: Luiz Felipe Scolari
Human: who was the captian of winning team
AI: Luiz Felipe Scolari
Human: who was the captian of winning team
AI: Luiz Felipe Scolari
Human: who was the captian of winning team
AI: Luiz Felipe Scolari


# ConersationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

convo = ConversationChain(llm =llm, memory=memory)

convo.run('who won first football world cup')
convo.run('what is 5 plus 5')
convo.run('who was the captian of winning team')
print(convo.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='who won first football world cup'), AIMessage(content='Brazil'), HumanMessage(content='what is 5 plus 5'), AIMessage(content='10'), HumanMessage(content='who was the captian of winning team'), AIMessage(content='Paulista')]) k=2
